In [3]:
import pandas as pd

In [4]:
clst_file = "/stor/work/Marcotte/project/rmcox/leca/ppi_ml/results/ppi_clustering/final_cmplx_annots_011823.csv"
outdir = "/stor/work/Marcotte/project/rmcox/leca/ppi_ml/results/cmplx_files/"

In [5]:
df = pd.read_csv(clst_file)
col_patterns = ['ID', 'cut', 'cmplx_name', 'characterization', 'human_gene', 'arath_gene']

cols = []
for p in col_patterns:
    res = df.filter(like=p)
    cols.append(res)
cols

fdf = pd.concat(cols, axis=1, ignore_index=False)
fdf[fdf['characterization_status'].notnull()]
fdf.head()

,ID,cut_2499.3,cut_2221.6,cut_1943.9,cut_1666.2,cut_1388.5,cut_1110.8,cut_833.1,cut_555.4,cut_277.7,...,corum_cmplx_name,cp_cmplx_name,granulated_cmplx_name_fixed,granulated_cmplx_name_unc,characterization_status,human_gene_names_primary,human_gene_names_synonym,human_gene_ontology_biological_process,arath_gene_names_primary,arath_gene_ontology_biological_process
0,ENOG502S254,140,206,339,473,621,791,982,1198,1440,...,NaN,NaN,NaN,NaN,Uncharacterized,NUDT1,MTH1,aging [GO:0007568]; DNA protection [GO:0042262...,NaN,NaN
1,KOG4146,140,206,339,473,621,791,982,1198,1440,...,NaN,NaN,NaN,NaN,Uncharacterized,URM1,C9orf74,protein urmylation [GO:0032447]; tRNA thio-mod...,"URM1-1, URM1-2",protein urmylation [GO:0032447]; tRNA thio-mod...
2,KOG1701,135,197,326,457,600,769,958,1173,1415,...,"Limd1-p62-Traf6-Prkcz complex, LPP-PPP2CA-PPP2...",NaN,NaN,NaN,Uncharacterized,"WTIP, TRIP6, ZYX, FBLIM1, LPP, AJUBA, LIMD1","OIP1, FBLP1, JUB",cell projection organization [GO:0030030]; cyt...,NaN,NaN
3,KOG2221,135,197,326,457,600,769,958,1173,1415,...,NaN,NaN,NaN,NaN,Uncharacterized,"TBC1D10B, TBC1D10C, TBC1D10A",EPI64 TBC1D10,activation of GTPase activity [GO:0090630]; in...,NaN,NaN
4,KOG2422,166,258,411,572,729,921,1128,1352,1602,...,NaN,Ribosome quality control complex,Ribosome quality control complex 2,NaN,Known,TCF25,KIAA1049 NULP1,heart development [GO:0007507]; negative regul...,NaN,NaN


In [43]:
counts = fdf.groupby(['granulated_cmplx_name']).size().sort_values(ascending=False)
counts[0:20]

granulated_cmplx_name
Uncharacterized complex 4                        86
Spliceosome 1                                    83
Uncharacterized complex 3                        63
Ribosomal cLSU                                   47
Ribosomal SSU processome 1                       44
Parvulin-associated pre-rRNP complex             36
Ribosomal cSSU                                   31
Ribosomal mLSU                                   29
Uncharacterized complex 5                        27
Mitochondrial respiratory chain complex I 1      25
RNA polymerase II complex                        25
SNARE complex 1                                  23
Uncharacterized complex 1                        23
19S Proteasome                                   20
IFT-B complex 1                                  20
Nuclear pore complex 3                           18
Aminoacyl-tRNA synthetase multienzyme complex    18
PRP19-associated complex                         18
eIF3 complex                              

In [46]:
cmplx_num = 0
for c, n in counts.items():
    if n > 1:
        cmplx_num += 1
    if n == 12:
        print(c)
cmplx_num

Uncharacterized emerin complex 2
Uncharacterized complex 7
Ribosomal mSSU
TRAPP complex


502

In [17]:
splice_cmplx = ['spliceosome', 'box', 'u2', 'snw1', 'swi', 'smn', 'prp']
splice_prots = []
known = []
novel = []
for i in range(len(fdf)):
    c = fdf['granulated_cmplx_name'][i]
    if any(x in str(c).lower() for x in splice_cmplx):
        splice_prots.append(c)
        if fdf['characterization_status'][i] == "Known":
            known.append(c)
        if fdf['characterization_status'][i] == "Novel member":
            novel.append(c)
len(splice_prots)

splice_prop = (len(splice_prots)/len(fdf))*100
print("# of splicing proteins = ", len(splice_prots))
print("-> # of known splicing proteins = ", len(known))
print("-> # of novel splicing proteins = ", len(novel))
print("% splicing proteins = ", splice_prop)

# of splicing proteins =  176
-> # of known splicing proteins =  130
-> # of novel splicing proteins =  40
% splicing proteins =  6.3354931605471565


In [38]:
!rm /stor/work/Marcotte/project/rmcox/leca/ppi_ml/results/cmplx_files/*
for c, n in counts.items():
    if n > 1:
        cdf = fdf[fdf['granulated_cmplx_name'] == c]
        c_fmt = c.replace(' ', '_')
        c_fmt = c_fmt.replace('(|)', '')
        c_fmt = c_fmt.replace('/', '-')
        outfile = outdir+c_fmt+'.csv'
        print(outfile)
        cdf.to_csv(outfile, index=False)

/stor/work/Marcotte/project/rmcox/leca/ppi_ml/results/cmplx_files/Uncharacterized_complex_4.csv
/stor/work/Marcotte/project/rmcox/leca/ppi_ml/results/cmplx_files/Spliceosome_1.csv
/stor/work/Marcotte/project/rmcox/leca/ppi_ml/results/cmplx_files/Uncharacterized_complex_3.csv
/stor/work/Marcotte/project/rmcox/leca/ppi_ml/results/cmplx_files/Ribosomal_cLSU.csv
/stor/work/Marcotte/project/rmcox/leca/ppi_ml/results/cmplx_files/Ribosomal_SSU_processome_1.csv
/stor/work/Marcotte/project/rmcox/leca/ppi_ml/results/cmplx_files/Parvulin-associated_pre-rRNP_complex.csv
/stor/work/Marcotte/project/rmcox/leca/ppi_ml/results/cmplx_files/Ribosomal_cSSU.csv
/stor/work/Marcotte/project/rmcox/leca/ppi_ml/results/cmplx_files/Ribosomal_mLSU.csv
/stor/work/Marcotte/project/rmcox/leca/ppi_ml/results/cmplx_files/Uncharacterized_complex_5.csv
/stor/work/Marcotte/project/rmcox/leca/ppi_ml/results/cmplx_files/Mitochondrial_respiratory_chain_complex_I_1.csv
/stor/work/Marcotte/project/rmcox/leca/ppi_ml/results/c